<a href="https://fabiandablander.com/r/Spike-and-Slab.html">Reference</a>

## Samples generation

We will now generate samples from a linear model

$$
\boldsymbol{y} = \boldsymbol{X}^T\boldsymbol{\beta} + \boldsymbol{\epsilon}
$$

where $\epsilon_i \sim \mathcal{N}(0,\sigma^2)$ and some of the coefficients $\beta_i=0$ in order to test our sparsity inducing bayesian model.

In [12]:
import numpy as np
import matplotlib.pyplot as plt

rng = np.random.RandomState(1234)
n_samples = 10

# X shaped like (n_samples,n_features)
X = rng.random(size=(n_samples,5))*10

# beta shaped like (n_features)
betas = np.array([
    1.,
    -2.,
    0.,
    4.,
    0.
    ])

sigma = 1

# y shaped like (n_samples)
y = X @ betas + rng.randn((n_samples))*sigma

# Gibbs Sampling the Posterior distribution for a linear model with Spike and Slab prior

Usual likelihood model for linear regression:

$$
P(y|\beta,x) \sim \mathcal{N}(<\beta , x>,\sigma^2)
$$


## Spike and Slab prior

We want to somehow give a certain probability for the coefficient $\beta$ to be exactly zero: that would be the case if its prior distribution was a $\delta (\beta)$. So we construct a prior that's a "mixture" of such delta function and some other prior distribution the allows the parameter to span a whole range of values:

<img src="./img/spikeslab.png" width=300>

The way this is usually implemented is by building a hierarchical model in which we "modulate" the prior based on an additional random variable $Z \sim Ber(\theta)$, that is $Z=1$ with probability $\theta$ and $Z=0$ with probability $1-\theta$. This additional variable regulates the prior distribution for $\beta$:

$$
P(Z_i) \sim Ber(\theta_i) \\
P(\beta_i|Z_i=0) \sim \delta(\beta_i) \\
P(\beta_i|Z_i=1) \sim P_{slab}(\beta_i)
$$

or more conveniently

$$
P(\beta_i|Z_i) = (1-Z_i)\delta(\beta_i) + Z_i P_{slab}(\beta_i)
$$

Where we subscripted the variables with $i$ because they are the coefficients in a linear regression model; since, if $Z_i=0$, the $\beta_i$ is inevitably zero, we can model the likelihood like this:

$$
P(y | \boldsymbol{z} , \boldsymbol{x}, \boldsymbol{\beta}) \sim \mathcal{N} (<\boldsymbol{z} \circ \boldsymbol{\beta},\boldsymbol{x}>,\sigma^2)
$$

Where $\boldsymbol{z}=(z_0,z_1,\dots)$ is the bernoulli vector and $\circ$ denotes the element-wise product. Also, we assume the slab distribution to be a gaussian centered at the origin, much like the ridge regression prior:

$$
P_{slab} = \mathcal{N} (0,\sigma^2 \tau^2)
$$

We put $\sigma^2$ in there because we want the distribution for $\beta$ to scale like the outcome.

We assume the following conjugate priors for the parameters:

$$
\theta \sim Beta(a,b) \\
\sigma^2 \sim InverseGamma(\alpha_1,\alpha_2) \\
\tau^2 \sim InverseGamma(1/2,s^2/2)
$$

It is useful to recap the relations between the random variables in a DAG, in order to properly express the conditional probabilities in our hierarchical model:

<img src="./img/DAG.png" width=400>

This helps us in factorizing the joint probability distribution via $d$-separation: (we do not subscript $\sigma^2$ and $\tau^2$ because we infer the same prior for every coefficient and same noise on each $y$)

$$
P(y,\beta_i,z_i,\theta_i,\tau^2,\sigma^2) = P(y|\beta_i,\sigma^2)P(\sigma^2)P(\beta_i|z_i,\tau^2)P(z_i|\theta_i)P(\theta_i)P(\tau^2)
$$

As for the hyperparameters in our priors (gray bubbles in graph), we fix $a=b=1$ for the beta distribution, $\alpha_1=\alpha_2=0.01$ for the $\sigma^2$ distribution and $s=1/2$ for the $\tau^2$ distribution.

The conditional posterior distribution can be broken up with the aid once again of the DAG shown above: 

$$
P(\theta_i | y,\beta_i,z_i,\tau^2,\sigma^2) = P(\theta_i|z_i) \\
P(\tau^2 | y,\beta_i,z_i,\theta_i,\sigma^2) = P(\tau^2 | \beta_i,z_i) \\
P(\sigma^2 | y,\beta_i,z_i,\theta_i,\tau^2) = P(\sigma^2 | y,\beta_i) \\
P(z_i | y,\beta_i,\theta_i,\tau^2,\sigma^2) = P(z_i | \beta_i,\theta_i,\tau^2) \\
P(\beta_i|y,\theta_i,z_i,\tau^2,\sigma^2) = P(\beta_i | y, z_i,\tau^2,\sigma^2)
$$

These conditional probabilities are needed to perform Gibbs sampling of the posterior distribution for all of these parameters. Let's derive their expressions.

### $P(\theta_i|z_i)$

Using Bayes' theorem:

$$
P(\theta_i|z_i) = \frac{P(z_i|\theta_i)P(\theta_i)}{\int P(z_i|\theta_i)P(\theta_i) d\theta_i}
$$

Now recall that $z_i \sim Ber(\theta_i) = \theta_i^{z_i}(1-\theta_i)^{1-z_i}$, and that we established the beta distribution with parameters $a,b$ as the prior for $\theta_i$:

$$
P(\theta_i|z_i) = \frac{\theta_i^{z_i}(1-\theta_i)^{1-z_i}\frac{1}{B(a,b)}\theta_i^{a-1}(1-\theta_i)^{b-1} }{ \int \theta_i^{z_i}(1-\theta_i)^{1-z_i}\frac{1}{B(a,b)}\theta_i^{a-1}(1-\theta_i)^{b-1} d\theta_i}
$$

which is again a Beta distribution normalized:

$$
P(\theta_i|z_i) = \frac{\theta_i^{(a+z_i)-1}(1-\theta_i)^{(b+1 - z_i)-1}}{\int \theta_i^{(a+z_i)-1}(1-\theta_i)^{(b+1 - z_i)-1}d\theta_i}
$$

so

$$
P(\theta_i|z_i) \sim Beta(a+z_i,b+1-z_i)
$$

### $P(\tau^2|\beta_i,z_i)$

This also has a dependency on $z_i$ when marginalizing because $P(\beta_i)$ has a dependency on both the parameters.

$$
P(\tau^2|\beta_i,z_i)= \frac{P(\beta_i|\tau^2,z_i)P(\tau^2)P(z_i)}{\int P(\beta_i|\tau^2,z_i)P(\tau^2)P(z_i) d\tau^2} = \frac{P(\beta_i|\tau^2,z_i)P(\tau^2)}{\int P(\beta_i|\tau^2,z_i)P(\tau^2) d\tau^2}
$$

We will denote here and later the normalization constant with $Z$. Now, the conditional probability on $\beta_i$ has a dependency on $z_i$: in particular, if 

### $z_i = 1$

$$
P(\tau^2 | \beta_i,z_i=1) = \frac{1}{Z}(2\pi\sigma^2\tau^2)^{-1/2}\exp \left [ -\frac{\beta_i^2}{2\sigma^2\tau^2} \right ] \frac{(s^2/2)^{1/2}}{\Gamma (1/2)}(\tau^2)^{-1/2 - 1}\exp \left [ -\frac{s^2/2}{\tau^2} \right ] = \frac{1}{Z}P(\beta|\tau^2,z_i=1)P(\tau^2)
$$

Which, absorbing every term not dependend on $\tau^2$ into the normalization constant, is again an Inverse Gamma distribution, which is expected since it's the conjugate prior for the exponential distribution with known mean:

$$
P(\tau^2|\beta_i, z_i=1) \sim InverseGamma(\frac{1}{2} + \frac{1}{2},\frac{s^2}{2}+\frac{\beta^2}{2\sigma^2})
$$

### $z_i = 0$

In this case, $\beta_i=0$ always and we simply sample from the prior:

$$
P(\tau^2|\beta_i, z_i=0) \sim InverseGamma(\frac{1}{2},\frac{s^2}{2})
$$


So we can summarize the $P(\tau^2|\beta_i,z_i)$ as 

$$
P(\tau^2|\beta_i,z_i) \sim InverseGamma(\frac{1}{2} + z_i \frac{1}{2},\frac{s^2}{2}+z_i\frac{\beta^2}{2\sigma^2})
$$


### $P(\sigma^2|y,\beta_i)$

Since $P(\sigma^2)\sim InverseGamma$, and from the likelihood probability of $\boldsymbol{y}$, we have

$$
\begin{align}
P(\sigma^2|y,\beta_i) &= \frac{1}{Z}P(y|\sigma^2,\beta_i)P(\sigma^2) \\
&= \frac{1}{Z} (2\pi\sigma^2)^{-n/2} \exp {\left[-\frac{1}{2\sigma^2} \sum_i^n (y_i - <\boldsymbol{\beta},\boldsymbol{x}_i>)^2 \right]}\frac{\alpha_2^{\alpha_1}}{\Gamma(\alpha_1)}(\sigma^2)^{-\alpha_1 -1 }\exp{\left[-\frac{\alpha_2}{\sigma^2}\right]}
\end{align}
$$

by absorbing every term that's not dependent on $\sigma^2$, we can rewrite this as

$$
P(\sigma^2|y,\beta_i) = \frac{1}{Z} (\sigma^2)^{-(\alpha_1 + \tfrac{n}{2})-1} \exp{ \left [ -\frac{1}{\sigma^2} \left( \alpha_2 + \frac{\sum_i^n (y_i-<\boldsymbol{\beta},\boldsymbol{x}_i>)^2}{2} \right ) \right ]}
$$

Which is again a Inverse Gamma, the conjugate prior for the gaussian likelihood: 

$$
P(\sigma^2|y,\beta_i) \sim Gamma \left ( \alpha_1 + \tfrac{n}{2},  \alpha_2 + \frac{\sum_i^n (y_i-<\boldsymbol{\beta},\boldsymbol{x}_i>)^2}{2} \right )
$$

Surprisingly, our posterior distribution for $\sigma^2$ gets updated with data points, but the distribution for $\tau^2$ does not, and instead is only dependent on the parameters $\beta$.

### $P(\beta|y,z,\tau^2,\sigma^2)$

We know that, when $z_i = 0$, the distribution collapses to the delta function centered at zero.
What about $z_i = 1$?

$$
\begin{align}
P(\beta|y,z=1,\tau^2,\sigma^2) & = \frac{P(y|\beta,z=1,\tau^2,\sigma^2)P(\beta|z=1)P(z=1)P(\tau^2)}{\int P(y|\beta,z=1,\tau^2,\sigma^2)P(\beta|z=1)P(z=1)P(\tau^2) d\beta} 
\end{align}
$$

By simplifying the terms that do not depend on $\beta$ and denoting with $Z$ again the normalization factor, we get

$$
P(\beta|y,z=1,\tau^2,\sigma^2) = \frac{1}{Z} (2\pi\sigma^2)^{-n/2} \exp {\left [ -\frac{1}{2\sigma^2} \sum_i^n(y_i - <\boldsymbol{\beta},\boldsymbol{x}_i>)^2 \right ]}(2\pi\sigma^2\tau^2)^{-1/2} \exp {\left [ -\frac{1}{2\sigma^2\tau^2} \beta \right ]}
$$

In [64]:
def ss_regress(
    y,
    X,
    a1 = 0.01,
    a2 = 0.01,
    theta = 0.5,
    a = 1,
    b = 1,
    s = 0.5,
    nr_samples = 6000,
    nr_burnin = None
):

    """
    X must be shaped like (n_data, n_features)
    """

    # ------------ PACKAGE IMPORTING
    from scipy.stats import beta, invgamma, multivariate_normal, binom


    n_data,n_features = X.shape

    if nr_burnin is None:
        nr_burnin = int(nr_samples/4)

    samples = np.empty((nr_samples,2*n_features+1+1+1)) # twice the number of coefficients to account for beta and the masking vector
                                                        # plus sigma plus tau plus theta
    
    beta0 = np.linalg.lstsq(X,y)[0] # get the initial guess for the coefficients from a least squares algorithm
    sigma0 = np.var(y- X @ beta0 ) # get the initial guess for sigma from the variance of the residuals
    tau0 = 1. # initial guess for tau
    theta0 = 0.5 # initial guess for theta

    XtX = X.T @ X # compute once
    Xty = X.T @ y # same

    samples[0,:n_features] = np.zeros(n_features) # initial guess of ones for the masking vector
    samples[0,n_features:2*n_features] = beta0 # initial guess for coefficients
    samples[0,-3] = sigma0
    samples[0,-2] = tau0
    samples[0,-1] = theta0

    for i in range(1,nr_samples):
        
        pi_prev = samples[i-1,:n_features]
        beta_prev = samples[i-1,n_features:2*n_features]
        sigma_prev = samples[i-1,-3]
        tau_prev = samples[i-1,-2]
        theta_prev = samples[i-1,-1]

        # sample theta from a Beta
        theta_new =  beta.rvs(a + np.sum(pi_prev),b+np.sum(1-pi_prev))
        # sample sigma2 from Inverse Gamma
        residuals = y - X @ beta_prev
        sigma_new = invgamma.rvs(a1 + n_data/2,a2 + (residuals.T @ residuals)/2)
        # sample tau2 from inverse gamma
        tau_new = invgamma.rvs(
            0.5+0.5*np.sum(pi_prev),
            (s**2)/2 + (beta_prev.T @ beta_prev)/(2*sigma_new)
            )
        # sample beta from multivariate gaussian
        covariance = np.linalg.inv( (  XtX/sigma_new + np.eye(XtX.shape[0])/(sigma_new*tau_new)  )  )
        mean = covariance @ Xty / (sigma_new)
        beta_new = multivariate_normal.rvs(mean,covariance)

        # sample each pi_j in random order

        for j in np.random.permutation(n_features):

            # get the betas for which beta_j is zero
            pi0 = pi_prev
            pi0[j] = 0 
            bp0 = beta_new * pi0 # sarebbe beta_{-j}


            # compute the z variables and the conditional variance
            xj = X[:,j]
            z = y - X @ bp0
            cond_var = np.sum(xj**2) + 1/tau_new

            # compute chance parameter of the conditional posterior of pi_j
            l0 = np.log(1-theta_new)
            l1 = np.log(theta_new) \
                - 0.5*np.log(tau_new*sigma_new)  \
                + (np.sum(xj*z)**2)/(2*sigma_new*cond_var) \
                + 0.5*np.log(sigma_new/cond_var)

            e_l0 = np.exp(l0) # this will always be between 0 and 1
            e_l1 = np.exp(l1) # this can be large; if that happens

            if True:
                if np.isinf(e_l1):
                    prob = 0
                else:
                    if np.isinf(e_l1+e_l0):
                        prob = 0
                    else:
                        print(e_l0)
                        print(e_l1)
                        print(np.isinf(e_l1))
                        print(l1)
                        prob = e_l0/(e_l0 + e_l1)


            # sample pi_j from a bernoulli
               
            
            print("prob: ",prob)
            pi_prev[j] = binom.rvs(1,   prob  )

        pi_new = pi_prev
        
        samples[i,:n_features] = pi_new
        samples[i,n_features:2*n_features] = beta_new
        samples[i,-3] = sigma_new
        samples[i,-2] = tau_new
        samples[i, -1] = theta_new

    return samples[nr_burnin:]
 

In [65]:
samples=ss_regress(y,X)

# np.isinf(np.exp(342435))

<ipython-input-64-56fb862d8dde>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  beta0 = np.linalg.lstsq(X,y)[0] # get the initial guess for the coefficients from a least squares algorithm
<ipython-input-64-56fb862d8dde>:90: RuntimeWarning: overflow encountered in exp
  e_l1 = np.exp(l1) # this can be large; if that happens


prob:  0
prob:  0
prob:  0
prob:  0
prob:  0
0.9951253681863543
5.72775502234833e+127
False
294.1736304705428
prob:  1.7373741794186608e-128
0.9951253681863543
3.049168963533147e+74
False
171.506165964079
prob:  3.263595360203582e-75
0.9951253681863543
1.2621126209111455e+79
False
182.13700934669166
prob:  7.884600404898514e-80
0.9951253681863543
8.593002540203572e+65
False
151.81897925855714
prob:  1.1580647899619723e-66
0.9951253681863543
4.7001968760739015e+104
False
241.01645406774583
prob:  2.1171993310577824e-105
0.8667520760823587
90232759412003.84
False
32.13341366344677
prob:  9.60573611768593e-15
0.8667520760823587
7.1277309142470605e+22
False
52.62086498479844
prob:  1.2160280550853515e-23
0.8667520760823587
7769534523990058.0
False
36.58898665067337
prob:  1.1155778681542386e-16
0.8667520760823587
1.1819098545366747e+17
False
39.311078231776946
prob:  7.333487175484613e-18
0.8667520760823587
1.902987971705504e+28
False
65.11580787142606
prob:  4.5546902501204694e-29
0.65099

<ipython-input-64-56fb862d8dde>:86: RuntimeWarning: overflow encountered in double_scalars
  + (np.sum(xj*z)**2)/(2*sigma_new*cond_var) \
<ipython-input-64-56fb862d8dde>:86: RuntimeWarning: invalid value encountered in double_scalars
  + (np.sum(xj*z)**2)/(2*sigma_new*cond_var) \


ValueError: Domain error in arguments.

In [19]:
from scipy.stats import binom
binom.rvs(1,2)

ValueError: Domain error in arguments.